In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic

# a = np.arange(0, 35)
# ic(a % 12)
# ic(a % 13)
# exit()

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
# six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
# six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    ic(name)
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)

# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)]=(matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0

matrix['delta_price_lag']=matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

ic| name: '!Якутск Орджоникидзе, 56 фран'
ic| name: '!Якутск ТЦ "Центральный" фран'
ic| name: 'Адыгея ТЦ "Мега"'
ic| name: 'Балашиха ТРК "Октябрь-Киномир"'
ic| name: 'Волжский ТЦ "Волга Молл"'
ic| name: 'Вологда ТРЦ "Мармелад"'
ic| name: 'Воронеж (Плехановская, 13)'
ic| name: 'Воронеж ТРЦ "Максимир"'
ic| name: 'Воронеж ТРЦ Сити-Парк "Град"'
ic| name: 'Выездная Торговля'
ic| name: 'Жуковский ул. Чкалова 39м?'
ic| name: 'Жуковский ул. Чкалова 39м²'
ic| name: 'Интернет-магазин ЧС'
ic| name: 'Казань ТЦ "Бехетле"'
ic| name: 'Казань ТЦ "ПаркХаус" II'
ic| name: 'Калуга ТРЦ "XXI век"'
ic| name: 'Коломна ТЦ "Рио"'
ic| name: 'Красноярск ТЦ "Взлетка Плаза"'
ic| name: 'Красноярск ТЦ "Июнь"'
ic| name: 'Курск ТЦ "Пушкинский"'
ic| name: 'Москва "Распродажа"'
ic| name: 'Москва МТРЦ "Афи Молл"'
ic| name: 'Москва Магазин С21'
ic| name: 'Москва ТК "Буденовский" (пав.А2)'
ic| name: 'Москва ТК "Буденовский" (пав.К7)'
ic| name: 'Москва ТРК "Атриум"'
ic| name: 'Москва ТЦ "Ареал" (Беляево)'
ic| name: 'Москва 

In [2]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code,item_cnt_month_lag_1,...,date_item_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_2,date_item_city_avg_item_cnt_lag_3,date_item_city_avg_item_cnt_lag_6,date_item_city_avg_item_cnt_lag_12,delta_price_lag,month,days,item_shop_first_sale,item_first_sale
4488756,12,2,27,0.0,19,5,10,1,4,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.282715,0,31,12,12
4488757,12,2,30,0.0,40,11,4,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.483398,0,31,11,11
4488758,12,2,31,0.0,37,11,1,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.137451,0,31,11,11
4488759,12,2,32,1.0,40,11,4,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.407227,0,31,12,12
4488760,12,2,33,1.0,37,11,1,1,4,1.0,...,1.0,2.0,0.0,0.0,1.0,-0.225464,0,31,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2,21,4,1.0,...,0.5,0.0,0.0,0.0,0.0,-0.475098,10,30,11,11
11128046,34,45,16188,0.0,64,14,42,21,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.081116,10,30,2,2
11128047,34,45,15757,0.0,55,13,2,21,4,0.0,...,0.0,0.5,0.0,0.0,0.0,0.155884,10,30,34,34
11128048,34,45,19648,0.0,40,11,4,21,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.091736,10,30,11,11


# 建模

In [10]:
trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

In [11]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)

In [12]:
params = {'objective': 'regression', 'metric': 'rmse', 'n_estimators': 1000, 'num_leaves': 200, 'learning_rate': 0.01, 'bagging_fraction': 0.9, 'feature_fraction': 0.3, 'bagging_seed': 0, 'early_stop_rounds': 50}

In [17]:
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.242558 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10805
[LightGBM] [Info] Number of data points in the train set: 6186922, number of used features: 63
[LightGBM] [Info] Start training from score 0.288852
[1]	training's rmse: 1.18283	valid_1's rmse: 1.13262
[2]	training's rmse: 1.17757	valid_1's rmse: 1.12866
[3]	training's rmse: 1.17357	valid_1's rmse: 1.12527
[4]	training's rmse: 1.16976	valid_1's rmse: 1.12263
[5]	training's rmse: 1.16472	valid_1's rmse: 1.11879
[6]	training's rmse: 1.16109	vali

[137]	training's rmse: 0.869926	valid_1's rmse: 0.930106
[138]	training's rmse: 0.868991	valid_1's rmse: 0.929619
[139]	training's rmse: 0.868175	valid_1's rmse: 0.92922
[140]	training's rmse: 0.867385	valid_1's rmse: 0.928868
[141]	training's rmse: 0.866558	valid_1's rmse: 0.928469
[142]	training's rmse: 0.865868	valid_1's rmse: 0.928056
[143]	training's rmse: 0.864782	valid_1's rmse: 0.927573
[144]	training's rmse: 0.864002	valid_1's rmse: 0.927312
[145]	training's rmse: 0.863154	valid_1's rmse: 0.926869
[146]	training's rmse: 0.862393	valid_1's rmse: 0.926482
[147]	training's rmse: 0.861693	valid_1's rmse: 0.926136
[148]	training's rmse: 0.861001	valid_1's rmse: 0.925949
[149]	training's rmse: 0.860197	valid_1's rmse: 0.925776
[150]	training's rmse: 0.859419	valid_1's rmse: 0.925452
[151]	training's rmse: 0.858858	valid_1's rmse: 0.925169
[152]	training's rmse: 0.858105	valid_1's rmse: 0.924538
[153]	training's rmse: 0.857447	valid_1's rmse: 0.924264
[154]	training's rmse: 0.85703	v

[282]	training's rmse: 0.800164	valid_1's rmse: 0.913745
[283]	training's rmse: 0.799961	valid_1's rmse: 0.913771
[284]	training's rmse: 0.799777	valid_1's rmse: 0.913692
[285]	training's rmse: 0.799596	valid_1's rmse: 0.913694
[286]	training's rmse: 0.799225	valid_1's rmse: 0.913634
[287]	training's rmse: 0.798972	valid_1's rmse: 0.913623
[288]	training's rmse: 0.798616	valid_1's rmse: 0.913952
[289]	training's rmse: 0.798372	valid_1's rmse: 0.913834
[290]	training's rmse: 0.798149	valid_1's rmse: 0.913845
[291]	training's rmse: 0.797971	valid_1's rmse: 0.913792
[292]	training's rmse: 0.797779	valid_1's rmse: 0.913834
[293]	training's rmse: 0.797477	valid_1's rmse: 0.913785
[294]	training's rmse: 0.7972	valid_1's rmse: 0.913738
[295]	training's rmse: 0.797039	valid_1's rmse: 0.913739
[296]	training's rmse: 0.796679	valid_1's rmse: 0.913726
[297]	training's rmse: 0.796514	valid_1's rmse: 0.913664
[298]	training's rmse: 0.796227	valid_1's rmse: 0.913631
[299]	training's rmse: 0.79591	va

[427]	training's rmse: 0.772765	valid_1's rmse: 0.913446
[428]	training's rmse: 0.772597	valid_1's rmse: 0.913429
[429]	training's rmse: 0.772422	valid_1's rmse: 0.913363
[430]	training's rmse: 0.772223	valid_1's rmse: 0.913309
[431]	training's rmse: 0.772127	valid_1's rmse: 0.913303
[432]	training's rmse: 0.771971	valid_1's rmse: 0.913332
[433]	training's rmse: 0.771771	valid_1's rmse: 0.913323
[434]	training's rmse: 0.771607	valid_1's rmse: 0.913264
[435]	training's rmse: 0.771494	valid_1's rmse: 0.913258
[436]	training's rmse: 0.77139	valid_1's rmse: 0.913253
[437]	training's rmse: 0.771278	valid_1's rmse: 0.91326
[438]	training's rmse: 0.771059	valid_1's rmse: 0.913261
[439]	training's rmse: 0.770948	valid_1's rmse: 0.913246
[440]	training's rmse: 0.77085	valid_1's rmse: 0.913293
[441]	training's rmse: 0.770739	valid_1's rmse: 0.913258
[442]	training's rmse: 0.770599	valid_1's rmse: 0.913298
[443]	training's rmse: 0.770479	valid_1's rmse: 0.91329
[444]	training's rmse: 0.77036	vali

[572]	training's rmse: 0.756598	valid_1's rmse: 0.913563
[573]	training's rmse: 0.756527	valid_1's rmse: 0.913557
[574]	training's rmse: 0.756466	valid_1's rmse: 0.913553
[575]	training's rmse: 0.756391	valid_1's rmse: 0.913559
[576]	training's rmse: 0.756313	valid_1's rmse: 0.913574
[577]	training's rmse: 0.75622	valid_1's rmse: 0.913469
[578]	training's rmse: 0.756173	valid_1's rmse: 0.913463
[579]	training's rmse: 0.756114	valid_1's rmse: 0.913474
[580]	training's rmse: 0.756024	valid_1's rmse: 0.913502
[581]	training's rmse: 0.755937	valid_1's rmse: 0.913515
[582]	training's rmse: 0.755784	valid_1's rmse: 0.913444
[583]	training's rmse: 0.755704	valid_1's rmse: 0.913453
[584]	training's rmse: 0.755642	valid_1's rmse: 0.913461
[585]	training's rmse: 0.755567	valid_1's rmse: 0.913428
[586]	training's rmse: 0.755502	valid_1's rmse: 0.913416
[587]	training's rmse: 0.755158	valid_1's rmse: 0.913334
[588]	training's rmse: 0.755106	valid_1's rmse: 0.913334
[589]	training's rmse: 0.754972	

[717]	training's rmse: 0.745644	valid_1's rmse: 0.91407
[718]	training's rmse: 0.74557	valid_1's rmse: 0.914097
[719]	training's rmse: 0.745494	valid_1's rmse: 0.914102
[720]	training's rmse: 0.745445	valid_1's rmse: 0.914095
[721]	training's rmse: 0.745397	valid_1's rmse: 0.914089
[722]	training's rmse: 0.745313	valid_1's rmse: 0.914068
[723]	training's rmse: 0.74524	valid_1's rmse: 0.914095
[724]	training's rmse: 0.745188	valid_1's rmse: 0.914082
[725]	training's rmse: 0.74514	valid_1's rmse: 0.914085
[726]	training's rmse: 0.745064	valid_1's rmse: 0.91407
[727]	training's rmse: 0.744991	valid_1's rmse: 0.914088
[728]	training's rmse: 0.744932	valid_1's rmse: 0.914081
[729]	training's rmse: 0.744874	valid_1's rmse: 0.914079
[730]	training's rmse: 0.744528	valid_1's rmse: 0.914077
[731]	training's rmse: 0.744478	valid_1's rmse: 0.914084
[732]	training's rmse: 0.744431	valid_1's rmse: 0.914087
[733]	training's rmse: 0.744386	valid_1's rmse: 0.914072
[734]	training's rmse: 0.744305	vali

[862]	training's rmse: 0.735883	valid_1's rmse: 0.914557
[863]	training's rmse: 0.735623	valid_1's rmse: 0.914476
[864]	training's rmse: 0.735568	valid_1's rmse: 0.914539
[865]	training's rmse: 0.735525	valid_1's rmse: 0.914533
[866]	training's rmse: 0.735469	valid_1's rmse: 0.914548
[867]	training's rmse: 0.73543	valid_1's rmse: 0.91455
[868]	training's rmse: 0.735384	valid_1's rmse: 0.914555
[869]	training's rmse: 0.735343	valid_1's rmse: 0.914545
[870]	training's rmse: 0.735304	valid_1's rmse: 0.914535
[871]	training's rmse: 0.735238	valid_1's rmse: 0.914493
[872]	training's rmse: 0.735189	valid_1's rmse: 0.914489
[873]	training's rmse: 0.735129	valid_1's rmse: 0.914487
[874]	training's rmse: 0.735042	valid_1's rmse: 0.914494
[875]	training's rmse: 0.734992	valid_1's rmse: 0.914514
[876]	training's rmse: 0.734939	valid_1's rmse: 0.914529
[877]	training's rmse: 0.734888	valid_1's rmse: 0.914476
[878]	training's rmse: 0.734841	valid_1's rmse: 0.914479
[879]	training's rmse: 0.734788	v

# test数据

In [18]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code,item_cnt_month_lag_1,...,date_item_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_2,date_item_city_avg_item_cnt_lag_3,date_item_city_avg_item_cnt_lag_6,date_item_city_avg_item_cnt_lag_12,delta_price_lag,month,days,item_shop_first_sale,item_first_sale
4488756,12,2,27,0.0,19,5,10,1,4,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.282715,0,31,12,12
4488757,12,2,30,0.0,40,11,4,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.483398,0,31,11,11
4488758,12,2,31,0.0,37,11,1,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.137451,0,31,11,11
4488759,12,2,32,1.0,40,11,4,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.407227,0,31,12,12
4488760,12,2,33,1.0,37,11,1,1,4,1.0,...,1.0,2.0,0.0,0.0,1.0,-0.225464,0,31,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0.0,55,13,2,21,4,1.0,...,0.5,0.0,0.0,0.0,0.0,-0.475098,10,30,11,11
11128046,34,45,16188,0.0,64,14,42,21,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.081116,10,30,2,2
11128047,34,45,15757,0.0,55,13,2,21,4,0.0,...,0.0,0.5,0.0,0.0,0.0,0.155884,10,30,34,34
11128048,34,45,19648,0.0,40,11,4,21,4,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.091736,10,30,11,11


In [19]:
testData = matrix[matrix['date_block_num'] == 34]
# label_test = testData['item_cnt_month']
X_test = testData.drop('item_cnt_month', axis=1)

# 预测

In [22]:
y_test = lgb_model.predict(X_test).clip(0, 20)
y_test

array([0.62966215, 0.41539438, 1.35523091, ..., 0.04768919, 0.02385558,
       0.03641442])

In [23]:
len(y_test)

214200

In [24]:
ss = pd.read_csv('./data/sample_submission.csv')
ss

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [28]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB


In [26]:
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})
submission

,ID,item_cnt_month
0,0,0.629662
1,1,0.415394
2,2,1.355231
3,3,0.525864
4,4,2.359572
...,...,...
214195,214195,0.088922
214196,214196,0.037873
214197,214197,0.047689
214198,214198,0.023856


In [30]:
submission.to_csv('./submit/sub2.csv', index=False)

In [29]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB
